In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from castep_parse import read_cell_file
import numpy as np

from utilities import decompress_file

In [3]:
def verify_castep_cell_with_atomistic_structure(cell_file, structure):
    """Verify the atoms and supercell listed in a CASTEP cell file are
    equivalent to those of an AtomisticStructure."""
    
    cst_cell = read_cell_file(cell_file)
        
    # Supercell:
    cst_supercell = cst_cell['supercell']
    struct_supercell = structure.supercell    
    assert np.allclose(cst_supercell, struct_supercell)
    
    # Atoms:
    struct_atoms_srt_idx = np.lexsort(np.round(structure.atoms.coords, decimals=9))
    struct_atoms = structure.atoms.coords[:, struct_atoms_srt_idx]
    
    
    cst_atoms_srt_idx = np.lexsort(np.round(cst_cell['atom_sites'], decimals=9))
    cst_atoms = cst_cell['atom_sites'][:, cst_atoms_srt_idx]
    
    # Wrap atoms to within the supercell:    
    struct_atoms_sup = np.linalg.inv(struct_supercell) @ struct_atoms
    cst_atoms_sup = np.linalg.inv(cst_supercell) @ cst_atoms
    
    boundary_idx = structure.boundary_idx
    struct_atoms_sup[boundary_idx] -= np.floor(struct_atoms_sup[boundary_idx])
    cst_atoms_sup[boundary_idx] -= np.floor(cst_atoms_sup[boundary_idx])
    
    bad_atoms_idx = np.where(np.abs(cst_atoms_sup - struct_atoms_sup) > 1e-6)[1]
    
    assert np.allclose(struct_atoms_sup, cst_atoms_sup)


## Ab-initio tensile tests

In [4]:
from tensile_test import get_tensile_test, show_traction_separation

**s7-tw-b-a1**

In [5]:
tt = get_tensile_test('s7-tw-b-a1')
test_input_structure = tt.get_coordinate(2.1).structure

In [6]:
cell_path_compressed = Path('data/ab_initio_tensile_tests/rgs-s7-tw-b-a1/16_+2.100/sim.cell.xz')
cell_file = decompress_file(cell_path_compressed)

In [7]:
verify_castep_cell_with_atomistic_structure(cell_file, test_input_structure)

**s7-tlA-gb-a1**

In [8]:
tt = get_tensile_test('s7-tlA-gb-a1')
test_input_structure = tt.get_coordinate(1.5).structure

In [9]:
cell_path_compressed = Path('data/ab_initio_tensile_tests/rgs-s7-tlA-gb-a1/03_+1.500/sim.cell.xz')
cell_file = decompress_file(cell_path_compressed)

In [10]:
verify_castep_cell_with_atomistic_structure(cell_file, test_input_structure)